In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 12.56 GB
Used RAM: 2.59 GB
Percentage Usage Of RAM: 18.8%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 15.78 GB
Used Disk: 9.20 GB
Percentage Usage Of Disk: 36.8%


In [3]:
# import requirements
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from gbm_classifier import Classification

[nltk_data] Downloading package vader_lexicon to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# get the data
star = pd.read_csv("star.csv")

In [5]:
# prepare the data for machine learning
star = star.drop(columns=["obj_ID", "rerun_ID", "spec_obj_ID"])
star = star.sample(frac=1, random_state=0).reset_index(drop=True)

In [6]:
# get the testing data
y = star[["class"]]
X = star.drop(columns="class")
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [7]:
# build the model
print("\n---- Star Classification Analysis ----\n")
model = Classification(
    name="XGBoost Without Feature Engineering", 
    path=None,
    rename=False, 
    time=False, 
    text=False,
    binary=False, 
    imputation=False, 
    variance=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    tune=True,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.explore(star)
    model.validate(X, y)  # build the machine learning pipeline
    predictions = model.predict(testX)
    print("\nModel Performance:")
    print(f"Accuracy: {model.accuracy}")
    print(f"F1: {model.f1}")
    print(f"In Control: {model.in_control}")
    print("Confusion Matrix:")
    pd.set_option("display.width", 1000)
    print(model.confusion / model.confusion.sum().sum())


---- Star Classification Analysis ----

Visualizing The Data:
> Plotting Correlations
> u vs. g
> u vs. z
> g vs. z
> r vs. i
> plate vs. MJD
> Plotting alpha
> Plotting delta
> Plotting u
> Plotting g
> Plotting r
> Plotting i
> Plotting z
> Plotting run_ID
> Plotting field_ID
> Plotting redshift
> Plotting plate
> Plotting MJD
> Plotting fiber_ID
> Plotting class
> alpha vs. cam_col
> delta vs. class
> delta vs. cam_col
> i vs. class
> z vs. class
> run_ID vs. cam_col
> plate vs. class
> fiber_ID vs. class
> fiber_ID vs. cam_col
49.86 Seconds
Model Training:
> Transforming The Training Data
> Removing Constant Features
> Tuning XGBoost
> Cross Validating 15 Models
> Training The Best Model
14.44 Minutes
Model Performance:
> Transforming The Testing Data
> Scoring The Model
31.65 Seconds
Model Indicators:
> Extracting Important Features
2.69 Seconds
Model Prediction:
> Transforming The New Data
> Getting Predictions
0.42 Seconds
Model Monitoring:
> Computing Feature Drift
0.87 Second

In [13]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10].tolist()):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10].tolist()):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. redshift
2. g
3. plate
4. z
5. u
6. r
7. i
8. MJD
9. run_ID
10. alpha
 
Feature Drift:
None


In [9]:
# score the model
accuracy = accuracy_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)
f1 = f1_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    average="macro",
)

print(f"Accuracy: {accuracy}")
print(f"F1: {f1}")

Accuracy: 0.981
F1: 0.9777266963185002


In [10]:
# show the confusion matrix
ytest = testy.iloc[:,0].to_numpy()
labels = np.unique(np.concatenate((predictions, ytest)))
confusion = confusion_matrix(
    y_true=ytest,   # rows
    y_pred=predictions,  # columns
    labels=labels,
)
confusion = pd.DataFrame(
    confusion, 
    columns=labels, 
    index=labels,
)
print("Confusion Matrix:")
pd.set_option("display.width", 1000)
print(confusion / confusion.sum().sum())

Confusion Matrix:
         GALAXY      QSO     STAR
GALAXY  0.58830  0.00680  0.00075
QSO     0.01145  0.17635  0.00000
STAR    0.00000  0.00000  0.21635


In [11]:
# save the machine learning pipeline
model.dump()

In [12]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Removing Constant Features
> Tuning XGBoost
> Cross Validating 15 Models
> Training The Best Model
9.2 Minutes
Model Indicators:
> Extracting Important Features
3.8 Seconds
